In [1]:
import cv2
from matplotlib import pyplot  as plt
import numpy as np
from math import cos 
from skimage.filters import (threshold_otsu, threshold_niblack,
                             threshold_sauvola)

In [2]:
def rescaling(image):
    max_dimension = max(image.shape)
    scale = 500.0/max_dimension 
    if scale < 1.0: 
        image = cv2.resize(image,None,fx=scale,fy=scale,interpolation=cv2.INTER_AREA )
        return image
    else:
        image = cv2.resize(image,None,fx=scale,fy=scale,interpolation = cv2.INTER_CUBIC)
        return image

In [ ]:
import glob
images = [rescaling(cv2.imread(file)) for file in glob.glob("./Desktop/test_images/*")]

In [3]:
def printImage(img):
    cv2.imshow('image',img)
    key = cv2.waitKey(0)
    cv2.destroyAllWindows()
    return key

In [ ]:
def printAndCompare(image1,image2):
    concatenateHorizontal = np.concatenate((image1,image2),axis = 1)
    cv2.imshow('concatenateHor',concatenateHorizontal)
    k=cv2.waitKey(0)
    cv2.destroyAllWindows()
    return k

In [ ]:
def GrayScale(img):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
    return gray

In [ ]:
def masking(img,sat,val):
    image_hsv = cv2.cvtColor(cv2.cvtColor(img,cv2.COLOR_GRAY2BGR),cv2.COLOR_BGR2HSV)
    min_val = np.array([0,0,0],np.uint8)
    max_val = np.array([255,sat,val],np.uint8)
    # masking out the image(separating bg from fg) on basis of values lying in the above range of hsv color channel
    mask = cv2.inRange(image_hsv,min_val,max_val)
    #inverting the mask
    invert_masked = cv2.bitwise_not(mask)
    #boolean anding with the original image
    invertMaskedColored = cv2.cvtColor(invert_masked,cv2.COLOR_GRAY2RGB)
    maskedImage = cv2.bitwise_and(image,invertMaskedColored)
    printAndCompare(maskedImage,image)
    #add maskedImage and invertMasked(bg px with 0 and fg px with 1) image to normalize fg px values
    finalMasked = cv2.addWeighted(maskedImage,1,cv2.cvtColor(invert_masked,cv2.COLOR_GRAY2BGR),0,1)
    return finalMasked

In [ ]:
def adaptiveThresh(img):
    thresh = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,5,2)
    printImage(thresh)

In [ ]:
def otsuThresh(img,kernelSize): 
    blur = cv2.GaussianBlur(img,(5,5),0)
    ret2,th2 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)    # convert to grayscale(binary image)
    return th2

In [ ]:
def sauvolaThreshold(img,_k,_r):
    thresh_sauvola = threshold_sauvola(img, window_size=15 ,k=_k,r=_r)
    binary_sauvola = img > thresh_sauvola
    plt.imsave('Sauvola',np.uint8(binary_sauvola)*255,cmap = 'gray',format = 'png')
    return plt.imread('Sauvola')

In [ ]:
def cannyEdge(image,threshold1,threshold2):
    edged = cv2.Canny(image, threshold1, threshold2)
    return edged

In [ ]:
# thresholding and finding contours
for ix,image in enumerate(images):
    #histogram equalization of high or low contrast image
    gray = GrayScale(image)
    clahe1 = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(6,6))
#     clahe2 = cv2.createCLAHE(clipLimit=6.0, tileGridSize=(8,8))
    cl1 = clahe1.apply(gray)
#     cl2 = clahe2.apply(gray)
#     k = printAndCompare(cl1,cl2)
    maskedImage = masking(cl1,120,100)
    k = printImage(maskedImage)
    if k==27:
        continue
    thresh = sauvolaThreshold(maskedImage,0.5,120)
    edged = cannyEdge(thresh,30,80)
    k = printImage(edged)
    if k==27:
        continue
    contouredIm,contours,hierarchy_arr = cv2.findContours(edged.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    hierarchy = hierarchy_arr[0]
#   contour_list = []
#   parentMap = {}
#   getMarkerList(0,1)
#   contour_list = np.array(contour_list)
    edged = cv2.cvtColor(edged,cv2.COLOR_GRAY2BGR)
    idx = 0
#     drawing all the contours at same hierarchy level with different colors 
    while idx >= 0:
        color = np.array([ np.random.randint(0,255) , np.random.randint(0,255), np.random.randint(0,255)])
        final_image = cv2.drawContours(image, np.array(contours), idx, color ,3 )
        k = printAndCompare(final_image,edged)
        if k==27:
            break
        idx = hierarchy[idx][0]
    concatenated = np.concatenate((final_image,edged),axis = 1)    
    imageName = 'contoured' + str(ix) +'.png'
    cv2.imwrite(imageName,concatenated)    


In [ ]:
#using hierarchy tree returns -1 or level to the calling parent and checking if the difference of the childLevel and parent level is 2
def getMarkerList(index,level):
    #if last child in hierarchy tree
    if hierarchy[index][0]==-1 and hierarchy[index][1]==-1 and hierarchy[index][2]==-1 and hierarchy[index][3]!=-1 :
        return level
    
    if hierarchy[index][0]==-1 and hierarchy[index][2]==-1 and hierarchy[index][1]!=-1:
        return -1
    #if child exists
    if hierarchy[index][2]!=-1 :
        childLevel = getMarkerList(hierarchy[index][2],level+1)  #explore its child hierarch
        #if level diff is less than 2 and next hierarchy component does not exists
        if childLevel!=-1:
            #analysing child
            if (childLevel-level)<2:
                if hierarchy[index][0] == -1 and hierarchy[index][1]==-1:
#                     print "-------------------------",level-childLevel
                    return childLevel
                elif hierarchy[index][0]!=-1:
                    return getMarkerList(hierarchy[index][0],level)
            if(childLevel-level)==2:
                #add to the cont_list and check next hierarchy level if exists return -1
                contour_list.append(contours[index])
                parentMap[index]=hierarchy[index][3] #contour index and its parent
                if hierarchy[index][0]!=-1:
                    return getMarkerList(hierarchy[index][0],level)
                return -1
        else:
            #child exists but contour value is not of interest
            if hierarchy[index][0]!=-1:
                return getMarkerList(hierarchy[index][0],level)
            return -1
    if hierarchy[index][0]!=-1:
        return getMarkerList(hierarchy[index][0],level)
       
    return -1    
    

In [ ]:
#contour_list is the all the components detected following the desired hierarchy
#further reduce the contour_list by checking ratio 
#parentMap consists of all the parent level hierarchy of Finder Pattern detected
new_list = []
contour_list = np.array(contour_list)
final_img = cv2.drawContours(image, contour_list,-1, (0,255,0), 3)
printImage(final_img)
cv2.destroyAllWindows()

NameError: name 'contour_list' is not defined

In [ ]:
# checking rectangle perimeter ratio and hierarchy
def SearchPattern(contour , hierarchy ,level , _dict ,cont_list):
    if(level>=2 and hierarchy[2].any()==-1) :
        level = level-1
        if isRect(contour) and SearchPattern(contours,hierarchy,level+1)==2:  #decrementing the level and ret value is 1
            _dict[level]==cv2.arcLength(contour)
            cont_list.append(contour)
            return 1
        else:
            return 0    
    peri = cv2.arcLength(contours,True)
    approx = cv2.approxPolyDP(contours,peri*0.02,True)
#     if  cv2.contourArea(contours) and len(approx)==4:
    if isRect(contour):
        ret = SearchPattern(contours[hierarchy[2]],hierarchy[hierarchy[2]],_dict,level+1 )
        if ret == 1:  
            level=level-1
            _dict[level] = cv2.arcLength(contours,True)
            cont_list.append()
            return 1
        if ret == 2:
            _dict[level] = cv2.arcLength(contours,True) 
            cont_list.append(contour)
            return 2
    else:
        return 0            

In [ ]:
#to check if the contour/shape is rectangle
def IsRect(contour):
    peri = cv2.arcLength(contour,True)
    approx = cv2.approxPolyDP(contour,peri*0.02,True)
    return  (cv2.contourArea(contour) and len(approx)==4) ##len(no of sides) of approximated curve is 4  

In [ ]:
def findPattern(contours, hierarchy):
    i=0
    contour_list =[]
    rect_count = 0
    while i<len(contours):
        if (hierarchy[i][1])==-1 and (hierarchy[i][0])==-1:
            i=hierarchy[contours[i]][2] 
        level= 0
        _dictionary = {}
        #if it is rectangle and child exists
        if IsRect(contours[i]) and ((hierarchy[i][2])==-1) :
            #check the value returned from the searchPattern to add contour to the list
            #if it is outer component(return value =2) go to 
            cont_list = []
            if SearchPattern(contours[hierarchy[i][2]],hierarchy[hierarchy[i][2]],level+1,_dictionary,cont_list)==2:
                _dictionary[level] = cv2.arcLength(contours[i])
                if _dictionary[0]/_dictionary[1] == 7.0/5.0 and _dictionary[1]/_dictionary[2] == 5.0/3.0:
                    contour_list.merge(cont_list.append(contours[i]))
                    rect_count = rect_count+1 
            if SearchPattern(contours[hierarchy[i][2]],hierarchy[hierarchy[i][2]],level+1,_dictionary,cont_list) == 1:
                    #check inner component if part of the pattern then  
                if _dictionary[0]/_dictionary[1] == 7.0/5.0 and _dictionary[1]/_dictionary[2] == 5.0/3.0:
                    contour_list.merge(cont_list)
                    rect_count+=1
                elif hierarchy[hierarchy[i][2]][1]!=-1:
                    i = hierarchy[hierarchy[i][2]][1] #go to the next level of its child
        else: 
            i = hierarchy[i][0]
    return (rect_count==3) , contour_list